In [2]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
from keras.layers import Input, Dense
from keras.models import Model, Sequential, load_model
from keras.optimizers import SGD, Adam
from keras.wrappers.scikit_learn import KerasRegressor
from keras import backend as K

/home/arkadii/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [3]:
data = pd.read_csv('data/four_data.csv')

data.describe()

,time,myo,aSX,aSY,aSZ,gSX,gSY,gSZ,aFX,aFY,aFZ,gFX,gFY,gFZ
count,4.886000e+04,48860.000000,48860.000000,48860.000000,48860.000000,48860.000000,48860.000000,48860.000000,48860.000000,48860.000000,48860.000000,48860.000000,48860.000000,48860.000000
mean,1.526620e+09,26.554482,52.066456,9.071832,8.259458,-0.127047,-0.159219,0.066309,0.066354,-2.031211,7.207546,-0.247855,-0.476253,0.044721
std,4.159146e+02,13.435690,134.170574,111.121119,48.986345,0.343420,0.434193,0.431138,149.351318,136.401449,69.995108,0.575854,0.553769,0.707733
min,1.526620e+09,4.000000,-321.112195,-324.404878,-280.226220,-1.478027,-1.480713,-1.610107,-470.842073,-494.622561,-340.720122,-2.029114,-2.029114,-1.708801
25%,1.526620e+09,17.000000,-28.344055,-40.382927,-19.982317,-0.262695,-0.466736,-0.117188,-60.603354,-63.707927,-16.045732,-0.580139,-0.695129,-0.609009
50%,1.526620e+09,24.000000,61.968902,-24.102439,10.749390,-0.133667,-0.141602,0.008057,-6.289634,0.438415,6.596951,-0.271545,-0.579163,0.423584
75%,1.526621e+09,32.000000,154.985366,90.884756,37.944512,0.007080,0.054810,0.263672,66.881098,48.121341,36.947256,-0.093079,-0.239563,0.500488
max,1.526621e+09,163.000000,304.558537,318.278049,237.212805,1.798462,2.250854,2.110229,380.377439,359.218902,332.429878,2.941345,2.291138,2.941345


In [7]:
from sklearn.preprocessing import MinMaxScaler, StandardScaler

X = data[[
 'aSX',
 'aSY',
 'aSZ',
 'gSX',
 'gSY',
 'gSZ',
 'myo'
]]


x_scaler = MinMaxScaler((0, 1))
x_scaler.fit(X)
x_scaled = x_scaler.transform(X)

X.describe()

,aSX,aSY,aSZ,gSX,gSY,gSZ,myo
count,48860.000000,48860.000000,48860.000000,48860.000000,48860.000000,48860.000000,48860.000000
mean,52.066456,9.071832,8.259458,-0.127047,-0.159219,0.066309,26.554482
std,134.170574,111.121119,48.986345,0.343420,0.434193,0.431138,13.435690
min,-321.112195,-324.404878,-280.226220,-1.478027,-1.480713,-1.610107,4.000000
25%,-28.344055,-40.382927,-19.982317,-0.262695,-0.466736,-0.117188,17.000000
50%,61.968902,-24.102439,10.749390,-0.133667,-0.141602,0.008057,24.000000
75%,154.985366,90.884756,37.944512,0.007080,0.054810,0.263672,32.000000
max,304.558537,318.278049,237.212805,1.798462,2.250854,2.110229,163.000000


In [8]:
Y = - data['gSY'] - data['gFY']

Y.describe()

count    48860.000000
mean         0.635472
std          0.789629
min         -4.269897
25%          0.386658
50%          0.602295
75%          1.019958
max          3.094055
dtype: float64

In [26]:
# Y = n_data['gyro_rf_y'] - n_data['gyro_rs_y']

In [75]:
# EMGX = n_data[[
#     'EMG_r', 
#     'acc_rs_x',
#     'acc_rs_y',
#     'acc_rs_z',
#     'gyro_rs_x',
#     'gyro_rs_y',
#     'gyro_rs_z',
# ]]

In [76]:
from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split(X, y_scaled, test_size=0.2)

In [118]:
simple_input = Input(shape=(3,))

output = Dense(12, activation='relu')(simple_input)
output = Dense(1, activation='sigmoid')(output)

model = Model(simple_input, output)

def root_mean_squared_error(y_true, y_pred):
    return K.sqrt(K.mean(K.square(y_pred - y_true)))

opt = SGD(lr=0.01, nesterov=True)

model.compile(optimizer=opt, loss='mean_squared_error',  metrics=[root_mean_squared_error])

In [119]:
from keras.callbacks import EarlyStopping, ReduceLROnPlateau, TensorBoard

learning_history = model.fit(X_train, Y_train,
                epochs=200,
                batch_size=256,
                shuffle=True,
                validation_split=0.2,
                callbacks=[
                    TensorBoard(log_dir='./logs/one_l_regression_2'),
                    ReduceLROnPlateau(verbose=1),
                    EarlyStopping(monitor='val_loss', min_delta=0.00001, patience=20, verbose=1, mode='auto')
                ])

Train on 1351596 samples, validate on 337899 samples
Epoch 1/200
1351596/1351596 [==============================] - 31s 23us/step - loss: 0.0034 - root_mean_squared_error: 0.0579 - val_loss: 0.0031 - val_root_mean_squared_error: 0.0550
Epoch 2/200
1351596/1351596 [==============================] - 30s 23us/step - loss: 0.0030 - root_mean_squared_error: 0.0544 - val_loss: 0.0030 - val_root_mean_squared_error: 0.0541
Epoch 3/200
1351596/1351596 [==============================] - 28s 21us/step - loss: 0.0029 - root_mean_squared_error: 0.0538 - val_loss: 0.0029 - val_root_mean_squared_error: 0.0537
Epoch 4/200
1351596/1351596 [==============================] - 25s 19us/step - loss: 0.0029 - root_mean_squared_error: 0.0536 - val_loss: 0.0029 - val_root_mean_squared_error: 0.0536
Epoch 5/200
1351596/1351596 [==============================] - 27s 20us/step - loss: 0.0029 - root_mean_squared_error: 0.0534 - val_loss: 0.0029 - val_root_mean_squared_error: 0.0535
Epoch 6/200
1351596/1351596 [===

KeyboardInterrupt: 

In [114]:
deep_input = Input(shape=(3,))

deep_model = Sequential()
deep_model.add(Dense(3, input_dim=3, activation='relu', kernel_initializer='normal'))
deep_model.add(Dense(6, activation='relu', kernel_initializer='normal'))
deep_model.add(Dense(12, activation='relu', kernel_initializer='normal'))
deep_model.add(Dense(3, activation='relu', kernel_initializer='normal'))
deep_model.add(Dense(1, kernel_initializer='normal'))

deep_opt = Adam(lr=0.01)

deep_model.compile(optimizer=deep_opt, loss='mean_squared_error', metrics=[root_mean_squared_error])

In [ ]:
learning_history = deep_model.fit(X_train, Y_train,
                epochs=200,
                batch_size=1024,
                shuffle=True,
                validation_split=0.1,
                callbacks=[
                    TensorBoard(log_dir='./logs/deep_regression_5'),
                    ReduceLROnPlateau(verbose=1),
                    EarlyStopping(monitor='val_loss', min_delta=0.00001, patience=20, verbose=1, mode='auto')
                ])